In [1]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes
!pip install -U openai
!pip install htmlrag

  Using cached bitsandbytes-0.45.5-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached bitsandbytes-0.45.5-py3-none-manylinux_2_24_x86_64.whl (76.1 MB)
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl (127.9 MB)
  Attempting uninstall: nvidia-cusolver-cu12
    Found existing installation: nvidia-cusolver-cu12 11.6.3.83
    Uninstalling nvidia-cusolver-cu12-11.6.3.83:
      Successfully uninstalled nvidia-cusolver-cu12-11.6.3.83
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.3/661.3 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.76.0
    Uninstalling openai-1.76.0:
      Successfully uninstalled openai-1.76.0
  Using cached

In [2]:
!git clone https://github.com/Uminchu618/lecture-ai-engineering.git

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 114 (delta 31), reused 26 (delta 13), pack-reused 62 (from 1)
Receiving objects: 100% (114/114), 570.45 KiB | 4.53 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# HuggingFace Login
from google.colab import userdata
from huggingface_hub import login

login(token=userdata.get('HUGGINGFACE_HUB_TOKEN'))

In [5]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
import random
random.seed(0)

In [7]:
# モデル(Llama3)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [8]:
# 質問のリスト
questions = [
    "すでに構築したクラウド環境を IaC に変換する機能はありますか？",
    "大量の機械学習トレーニングデータはどこに保管すべきですか？その理由も含めて説明してください。",
    "SageMakerのトレーニングジョブが S3 にアクセスするには IAM ユーザーが必要ですか？",
    "SageMakerのトレーニング費用を抑える方法はありますか？",
    "Notebook Instance と SageMaker Studio はどう使い分けるべきですか？",
    "BedrockでDeepSeek-R1を使った場合、どのくらいの推論コストがかかりますか？"
]

# 回答のリスト
answers = [
    "はい。AWS CloudFormation の IaC ジェネレーター機能では、既存リソースからテンプレートを生成することができます。Terraform でも `terraform import` コマンドを使い、状態ファイルに既存リソースを登録できます。",
    "Amazon S3 が推奨です。オブジェクトを自動で冗長化し、11 ナイン (99.999999999 %) の耐久性を提供します。データレイクや SageMaker とネイティブ統合され、ペタバイト級でもスケールを意識せずに利用できます。S3 イベントで Lambda を呼び出せば、取込直後の前処理やメタデータ登録も自動化できます。",
    "IAM ユーザーは不要です。SageMaker には **IAM 実行ロール** を付与し、そのロールに `s3:*` など必要なポリシーをアタッチします。ジョブ実行時、SageMaker はこのロールを引き受けて S3 や CloudWatch などのサービス API を呼び出します。",
    "Amazon SageMaker **Managed Spot Training** を有効にすると、オンデマンド比で最大 90 % のコスト削減が可能です。Spot 中断に備えてチェックポイントを S3 に保存する仕組みがあり、再開時に自動で引き継げます。学習時間よりコスト優先のワークロードで効果が高いベストプラクティスです。",
    "Studio Classic ノートブックは起動が Notebook Instance より 5–10 倍速く、インスタンス種別もブラウザから即変更できます。Notebook Instance はシングルユーザーで完全に分離した環境が欲しい場合や、従来の CLI/SDK 手順を重視するケースに向きます。いずれも従量課金ですが、Studio はアイドル状態の EFS ストレージ料金、Instance は EC2 実行料金の最小化がポイントです。",
    "1,000 入力トークンあたりの料金は 0.00135 USD、1,000 出力トークンあたりの料金は 0.0054 USDです。これは、他のオープンモデル（Llama 3.3 など）に対して比較的高い料金となっています。"
]


In [9]:
def generate_output(query, system_prompt=None):
  if system_prompt is None:
    messages = [
        {"role": "user", "content": query},
    ]
  else:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [10]:
# システムプロンプト
system_prompt = "質問に回答してください。必ず「日本語で回答」すること。１００文字以内で回答すること。"
responses = []
# 各質問に回答を生成
for i, q in enumerate(questions, start=1):
    print(f"### {i}. {q}")
    response = generate_output(q, system_prompt)
    print("**回答**")
    print(response)
    print("\n" + "---" + "\n")
    responses.append(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


### 1. すでに構築したクラウド環境を IaC に変換する機能はありますか？


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


**回答**
はい、すでに構築したクラウド環境を Infrastructure as Code (IaC) に変換する機能はあります。例えば、AWS CloudFormation、Azure Resource Manager (ARM)、Google Cloud Deployment Manager などのクラウドプロバイダーが提供するツールや、third-party のソフトウェアとして、Cloudify、Terraform、Ansible などがあります。これらのツールを使用することで、既に構築されたクラウド環境を IaC に変換し、自動化されたインフラストラクチャーの管理を実現することができます。

---

### 2. 大量の機械学習トレーニングデータはどこに保管すべきですか？その理由も含めて説明してください。


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

**回答**
はい、SageMakerのトレーニングジョブがS3にアクセスするには、IAMユーザーが必要です。SageMakerは、IAMを使用してS3にアクセスするための認証を行います。SageMakerのIAMロールには、S3にアクセスするための権限が必要です。特に、S3バケットに書き込みや読み取りを行う権限が必要です。SageMakerの設定で、IAMロールを指定することで、SageMakerがS3にアクセスするための認証を行うことができます。

---

### 4. SageMakerのトレーニング費用を抑える方法はありますか？


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


**回答**
はい、SageMakerのトレーニング費用を抑える方法はいくつかあります。以下はその一例です。

1. ハイパーパラメーターのチューニング：適切なハイパーパラメーターを選択することで、モデルがより効率的に学習することができます。
2. モデルの圧縮：モデルを圧縮することで、ストレージやネットワークの使用量を減らすことができます。
3. 並列化：複数のGPUやインスタンスに分散して計算を実行することで、トレーニング時間を短縮することができます。
4. 低コストのインスタンスの使用：低コストのインスタンスを使用することで、トレーニング費用を抑えることができます。
5.  Early Stopping：モデルが過学習するのを防ぐために、早期に停止することができます。

これらの方法を組み合わせることで、SageMakerのトレーニング費用を抑えることができます。

---

### 5. Notebook Instance と SageMaker Studio はどう使い分けるべきですか？


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


**回答**
Notebook InstanceとSageMaker Studioは、AWS SageMakerの2つのサービスです。Notebook Instanceは、Jupyter NotebookやApache Zeppelinなどのnotebook環境を提供し、データサイエンティストや機械学習エンジニアがコードを実行するための環境を提供します。SageMaker Studioは、統合された開発環境で、データサイエンティストや機械学習エンジニアが、データの探索、特徴量の作成、モデル開発、デプロイメントなど、機械学習の全プロセスを一元化して実行することができます。

Notebook Instanceは、個々のnotebook環境を提供し、各々のプロジェクトに特化した環境を提供します。SageMaker Studioは、複数のnotebook環境を統合し、プロジェクト全体のビューを提供します。したがって、Notebook Instanceは、個々のnotebook環境を提供し、SageMaker Studioは、プロジェクト全体のビューを提供するという使い分けが適切です。

---

### 6. BedrockでDeepSeek-R1を使った場合、どのくらいの推論コストがかかりますか？
**回答**
BedrockのDeepSeek-R1は、推論コストを最小化するために設計された。具体的には、推論コストは、DeepSeek-R1のモデルサイズ、推論の精度、ハードウェアの性能などに依存します。

しかし、Bedrockの公式ドキュメントによると、DeepSeek-R1の推論コストは、通常の推論コストの1/10から1/100程度に抑えられます。例えば、1秒あたり100万回の推論を実行する場合、DeepSeek-R1では1秒あたり1万回から10万回の推論を実行することができます。

ただし、実際の推論コストは、ハードウェアの性能、モデルサイズ、推論の精度などに依存するため、実際の推論コストは異なる可能性があります。

---



In [11]:
from openai import OpenAI

client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"), max_retries=5, timeout=60)

def openai_generator(query):

        messages = [
                    {
                        "role": "user",
                        "content": query
                    }
                ]

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        return response.choices[0].message.content

def evaluate_answer_accuracy(query, response, reference):

    template_accuracy1 = (
          "Instruction: You are a world class state of the art assistant for rating "
          "a User Answer given a Question. The Question is completely answered by the Reference Answer.\n"
          "Say 4, if User Answer is full contained and equivalent to Reference Answer"
          "in all terms, topics, numbers, metrics, dates and units.\n"
          "Say 2, if User Answer is partially contained and almost equivalent to Reference Answer"
          "in all terms, topics, numbers, metrics, dates and units.\n"
          "Say 0, if User Answer is not contained in Reference Answer or not accurate in all terms, topics,"
          "numbers, metrics, dates and units or the User Answer do not answer the question.\n"
          "Do not explain or justify your rating. Your rating must be only 4, 2 or 0 according to the instructions above.\n"
          "Even small discrepancies in meaning, terminology, directionality, or implication must result in a lower score. Only rate 4 if the User Answer is a complete and precise match to the Reference Answer in every aspect.\n"
          "### Question: {query}\n"
          "### {answer0}: {sentence_inference}\n"
          "### {answer1}: {sentence_true}\n"
          "The rating is:\n"
      )
    template_accuracy2 = (
          "I will rate the User Answer in comparison to the Reference Answer for a given Question.\n"
          "A rating of 4 indicates that the User Answer is entirely consistent with the Reference Answer, covering all aspects, topics, numbers, metrics, dates, and units.\n"
          "A rating of 2 signifies that the User Answer is mostly aligned with the Reference Answer, with minor discrepancies in some areas.\n"
          "A rating of 0 means that the User Answer is either inaccurate, incomplete, or unrelated to the Reference Answer, or it fails to address the Question.\n"
          "I will provide the rating without any explanation or justification, adhering to the following scale: 0 (no match), 2 (partial match), 4 (exact match).\n"
          "Even minor inconsistencies in meaning, terminology, emphasis, or factual detail should prevent a rating of 4. Only assign a 4 if the User Answer exactly and unambiguously matches the Reference Answer in every respect."
          "Do not explain or justify my rating. My rating must be only 4, 2 or 0 only.\n\n"
          "Question: {query}\n\n"
          "{answer0}: {sentence_inference}\n\n"
          "{answer1}: {sentence_true}\n\n"
          "Rating: "
      )

    score1 = openai_generator(
                template_accuracy1.format(
                      query=query,
                      answer0="User Answer",
                      answer1="Reference Answer",
                      sentence_inference=response,
                      sentence_true=reference,
                    )
                )
    try:
      score1 = int(score1)
    except:
      print("Failed")
      score1 = 0

    score2 = openai_generator(
                template_accuracy2.format(
                        query=query,
                        answer0="Reference Answer",
                        answer1="User Answer",
                        sentence_inference=reference,
                        sentence_true=response,
                    )
                  )

    try:
      score2 = int(score2)
    except:
      print("Failed")
      score2 = 0


    return (score1 + score2) / 2

In [12]:
print("| 評価番号 | スコア |")
print("|:--------:|:------:|")

# Iterate and print each row with only index and score
for i, (question, gold_answer, response) in enumerate(zip(questions, answers, responses), start=1):
    score = evaluate_answer_accuracy(question, response, gold_answer)
    print(f"| {i} | {score:.2f} |")


| 評価番号 | スコア |
|:--------:|:------:|
| 1 | 2.00 |
| 2 | 1.00 |
| 3 | 0.00 |
| 4 | 0.00 |
| 5 | 0.00 |
| 6 | 0.00 |


In [13]:
import htmlrag
import re
import glob

file_paths = glob.glob('/content/lecture-ai-engineering/day3/homework_data/*.html')
documents = []
for path in file_paths:
  html=open(path).read()
  simplified_html=htmlrag.clean_html(html)
  block_tree, simplified_html= htmlrag.build_block_tree(simplified_html, max_node_words=512)
  blocks = [
      re.sub(r'<[^>]+>', '', str(node[0])).split("\n")
      for node in block_tree
  ]
  documents.extend( [x for x in sum(blocks,[]) if len(x) > 20] )# 短い断片はゴミなので削除した


In [14]:
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[100])

ドキュメントサイズ:  1441
ドキュメントの例: 
 SageMaker AI コンソール、「https://https://console.aws.amazon.com/sagemaker/://www.


In [15]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 8192

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [16]:
responses = []
for i, question in enumerate(questions):
  print(f"### {i+1}. {question}")
  query_embeddings = emb_model.encode([question], prompt_name="query")
  document_embeddings = emb_model.encode(documents)

  # 各ドキュメントの類似度スコア
  scores = (query_embeddings @ document_embeddings.T) * 100
  topk = 5
  for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
    references = []
    for ref in ["\n".join(documents[max(0, i-1): min(i+1, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]]:

      system_prompt = "与えられた参考資料が質問に直接関連しているか？'yes''no'で答えること。ただし、余計なテキストを生成しないこと。"
      user_prompt =  f"[参考資料]\n{ref}\n\n[質問] {question}"
      response = generate_output(user_prompt, system_prompt)

      if "yes" in response.lower():
        print("\n\n対象となるドキュメント:\n", ref.replace("。", "。\n"))
        references.append(ref)

      print("\n\n")

  system_prompt = "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"
  user_prompt =  f"[参考資料]\n{references}\n\n[質問] {question}"
  response = generate_output(user_prompt, system_prompt)
  print("**回答**")
  print(response)
  print("\n" + "---" + "\n")
  responses.append(response)


### 1. すでに構築したクラウド環境を IaC に変換する機能はありますか？


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.




対象となるドキュメント:
 IaC ジェネレーターの機能は、すべての商用リージョンで利用可能で、多くの一般的な AWS リソースタイプをサポートしています。
サポートされているリソースの一覧については、「リソースタイプのサポート」を参照してください。
トピック考慮事項リソースのスキャンに必要な IAM アクセス許可テンプレートの生成、管理、削除によく使用されるコマンドテンプレートを AWS CDK に移行するCloudFormation IaC ジェネレーターを使用してリソーススキャンを開始するCloudFormation コンソールでスキャンの概要を表示するIaC ジェネレーターを使用してスキャンされたリソースから CloudFormation テンプレートを作成するスキャンしたリソースから CloudFormation スタックを作成する書き込み専用プロパティを解決する
読み取り権限のある AWS リソースであれば、JSON テンプレートまたは YAML テンプレートを生成できます。
IaC ジェネレーターの機能のテンプレートは、リソースプロパティをプロパティごとに記述することなく、クラウドリソースを確実かつ迅速にモデル化します。






対象となるドキュメント:
 読み取り権限のある AWS リソースであれば、JSON テンプレートまたは YAML テンプレートを生成できます。
IaC ジェネレーターの機能のテンプレートは、リソースプロパティをプロパティごとに記述することなく、クラウドリソースを確実かつ迅速にモデル化します。

次の表は、IaC の生成機能に使用できるクォータの一覧です。






The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.




対象となるドキュメント:
 料金の例 6: Bedrock データオートメーション標準出力を使用して、組織内の 15,000 分間の会議の音声録音を処理するとします。
15,000 分すべての音声処理にかかる合計コストは 90 USD です。
  合計処理時間 = 15,000 分 合計料金 = 15,000 分 × 0.006 USD = 90 USD
アプリケーションデベロッパーは、時間単位で、Amazon Bedrock に対して次の API コールを実行します: 入力テキストの 2K トークンの入力を 1K トークン (推論トークンを含む) の出力に要約する、DeepSeek-R1 モデルに対するリクエスト:








The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.





**回答**
参考資料に基づいて、BedrockでDeepSeek-R1を使用した場合、1Kトークンの出力に要約するために必要な推論コストは、0.006 USD per 2K input tokensです。

したがって、15,000分間の会議の音声録音を処理するために必要な推論コストは、15,000分 × 0.006 USD = 90 USDとなります。

---



In [17]:
print("| 評価番号 | スコア |")
print("|:--------:|:------:|")

# Iterate and print each row with only index and score
for i, (question, gold_answer, response) in enumerate(zip(questions, answers, responses), start=1):
    score = evaluate_answer_accuracy(question, response, gold_answer)
    print(f"| {i} | {score:.2f} |")

| 評価番号 | スコア |
|:--------:|:------:|
| 1 | 2.00 |
| 2 | 1.00 |
| 3 | 0.00 |
| 4 | 2.00 |
| 5 | 1.00 |
| 6 | 0.00 |


In [19]:
import csv

with open('documents.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['index', 'document'])
    for index, document in enumerate(documents):
        writer.writerow([index, document])